In [1]:
# Welcome to the Kalshi REST v2 Starter Code!

# pypi client: recommended for more advanced programmers
#import kalshi_python

# starter client: recommended for all levels of programming experience (what this client is implemented using)
from KalshiClientsBaseV2ApiKey import ExchangeClient
import time
import json
import uuid

In [2]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

def load_private_key_from_file(file_path):
    with open(file_path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,  # or provide a password if your key is encrypted
            backend=default_backend()
        )
    return private_key

In [ ]:
# To start off, you need to have created an account at https://kalshi.com (Production) 
# or an account on the Demo https://demo.kalshi.co/

prod_key_id = "2ff8666e-cf7b-4ff4-b667-06568a3f6b1b" # change these to be your personal credentials
prod_private_key = load_private_key_from_file('HybridNation-2.txt')

demo_key_id = "8b892002-a4ea-458b-9273-e09272353368" # change these to be your personal credentials
demo_private_key = load_private_key_from_file('HybridNation.txt')

# for prod
prod_api_base = "https://api.elections.kalshi.com/trade-api/v2"

# for demo
demo_api_base = "https://demo-api.kalshi.co/trade-api/v2"

## if wanting to test in prod:
exchange_client = ExchangeClient(exchange_api_base=prod_api_base, key_id = prod_key_id, private_key = prod_private_key)

## if wanting to test in demo
# exchange_client = ExchangeClient(exchange_api_base = demo_api_base, key_id = demo_key_id, private_key = demo_private_key)

# first we will check on the exchange status to confirm you are properly connected...
print(exchange_client.get_exchange_status())

TypeError: ExchangeClient.__init__() got an unexpected keyword argument 'host'

In [4]:
# You can discover markets through the get_markets endpoint...

# and use query parameters to filter your search!
market_params = {'limit':100,
                    'cursor':None, # passing in the cursor from the previous get_markets call
                    'event_ticker': None,
                    'series_ticker':None,
                    'max_close_ts':None, # pass in unix_ts
                    'min_close_ts':None, # pass in unix_ts
                    'status':None,
                    'tickers':None}

markets_response = exchange_client.get_markets(**market_params)
cursor = markets_response['cursor']

print('keys:', markets_response.keys())
print()
print('number of objects:', len(markets_response['markets'])) # 100 objects!
print()
print('first market in payload:', markets_response['markets'][0])
print()
print('cursor:', cursor)

keys: dict_keys(['markets', 'cursor'])

number of objects: 100

first market in payload: {'ticker': 'KXNHLGAME-25APR23MTLWSH-WSH', 'event_ticker': 'KXNHLGAME-25APR23MTLWSH', 'market_type': 'binary', 'title': 'Montreal vs Washington Winner?', 'subtitle': '', 'yes_sub_title': 'Washington', 'no_sub_title': 'Washington', 'open_time': '2025-04-22T02:20:00Z', 'close_time': '2027-04-23T23:00:00Z', 'expected_expiration_time': '2025-04-24T02:00:00Z', 'expiration_time': '2027-04-23T23:00:00Z', 'latest_expiration_time': '2027-04-23T23:00:00Z', 'settlement_timer_seconds': 500, 'status': 'initialized', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 0, 'yes_ask': 0, 'no_bid': 100, 'no_ask': 100, 'last_price': 0, 'previous_yes_bid': 0, 'previous_yes_ask': 0, 'previous_price': 0, 'volume': 0, 'volume_24h': 0, 'liquidity': 0, 'open_interest': 0, 'result': '', 'can_close_early': True, 'expiration_value': '', 'category': '', 'risk_limit_cents': 0, 'strike_type': 'st

In [5]:
# What are cursors and how do they work?
    
# The Cursor represents a pointer to the next page of records in the pagination.
# So this optional parameter, when filled, should be filled with the cursor string returned in a previous request to this end-point.
# Filling this would basically tell the api to get the next page containing the number of records passed on the limit parameter.
# On the other side not filling it tells the api you want to get the first page for another query.
# The cursor does not store any filters, so if any filter parameters like tickers, max_ts or min_ts were passed in the original query they must be passed again.

# Let's try it in action! Suppose we wanted to get the next 100 market objects...

market_params = {'limit':100,
                    'cursor':cursor, # passing in the cursor from the previous get_markets call
                    'event_ticker': None,
                    'series_ticker': None,
                    'max_close_ts': None, # pass in unix_ts
                    'min_close_ts': None, # pass in unix_ts
                    'status': None,
                    'tickers':None}

markets_response = exchange_client.get_markets(**market_params)
cursor = markets_response['cursor']

print('keys:', markets_response.keys())
print()
print('number of objects:', len(markets_response['markets'])) # 100 objects!
print()
print('first market in market_response payload:', markets_response['markets'][0]) # new markets!
print()
print('new cursor!', cursor)

keys: dict_keys(['markets', 'cursor'])

number of objects: 100

first market in market_response payload: {'ticker': 'KXSPOTIFY2D-25APR22-JUS', 'event_ticker': 'KXSPOTIFY2D-25APR22', 'market_type': 'binary', 'title': 'Will the top Song on Apr 22, 2025 be Just In Case?', 'subtitle': ':: Morgan Wallen', 'yes_sub_title': 'Just In Case', 'no_sub_title': 'Just In Case', 'open_time': '2025-04-21T21:00:00Z', 'close_time': '2025-04-23T03:59:00Z', 'expected_expiration_time': '2025-04-23T14:00:00Z', 'expiration_time': '2025-05-06T14:00:00Z', 'latest_expiration_time': '2025-05-06T14:00:00Z', 'settlement_timer_seconds': 1800, 'status': 'active', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 0, 'yes_ask': 1, 'no_bid': 99, 'no_ask': 100, 'last_price': 0, 'previous_yes_bid': 0, 'previous_yes_ask': 0, 'previous_price': 0, 'volume': 0, 'volume_24h': 0, 'liquidity': 1810, 'open_interest': 0, 'result': '', 'can_close_early': True, 'expiration_value': '', 'category':

In [6]:
# Next, let's look at event level data by passing an event ticker to the get_event endpoint...

event_ticker = markets_response['markets'][5]['event_ticker']
event_params = {'event_ticker': event_ticker}
event_response = exchange_client.get_event(**event_params)

print('keys:', event_response.keys())
print()
print('event object:', event_response['event'])
print()
print('first market in event_response payload:', event_response['markets'][0])

keys: dict_keys(['event', 'markets'])

event object: {'event_ticker': 'KXSPOTIFY2D-25APR22', 'series_ticker': 'KXSPOTIFY2D', 'sub_title': 'On Apr 22, 2025 Chart', 'title': 'Runner-up USA Song on Spotify on Apr 22, 2025?', 'collateral_return_type': 'MECNET', 'mutually_exclusive': True, 'category': 'Entertainment'}

first market in event_response payload: {'ticker': 'KXSPOTIFY2D-25APR22-NOK', 'event_ticker': 'KXSPOTIFY2D-25APR22', 'market_type': 'binary', 'title': '', 'subtitle': ':: Drake', 'yes_sub_title': 'NOKIA', 'no_sub_title': 'NOKIA', 'open_time': '2025-04-21T21:00:00Z', 'close_time': '2025-04-23T03:59:00Z', 'expected_expiration_time': '2025-04-23T14:00:00Z', 'expiration_time': '2025-05-06T14:00:00Z', 'latest_expiration_time': '2025-05-06T14:00:00Z', 'settlement_timer_seconds': 1800, 'status': 'active', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 0, 'yes_ask': 0, 'no_bid': 100, 'no_ask': 100, 'last_price': 0, 'previous_yes_bid': 0, 'previo

In [7]:
# Next, let's look at series level data by passing a series ticker to the get_series endpoint! 
series_ticker = event_response['event']['series_ticker']
series_params = {'series_ticker': series_ticker}
series_response = exchange_client.get_series(**series_params)

print('keys:', series_response.keys())
print()
print('series object:', series_response['series'])
print()

keys: dict_keys(['series'])

series object: {'ticker': 'KXSPOTIFY2D', 'frequency': 'daily', 'title': 'Daily US Spotify chart Runner-up', 'category': 'Entertainment', 'tags': ['Music charts'], 'settlement_sources': [{'url': 'https://charts.spotify.com/charts/view/regional-us-daily/latest', 'name': 'Spotify Charts'}], 'contract_url': 'https://kalshi-public-docs.s3.us-east-1.amazonaws.com/regulatory/product-certifications/SPOTIFYCHARTRANK.pdf'}



In [10]:
# Next let's look at the recent market history for a market
ticker = markets_response['markets'][0]['ticker']

market_history_params = {'ticker': ticker,
                            'limit': 100,
                            'cursor': None,
                            'max_ts': None, # pass in unix_ts
                            'min_ts': None # passing a recent unix_ts
                                }
market_history_response = exchange_client.get_market_history(**market_history_params)

print('keys:', market_history_response.keys())
print()
print('most recent market history object:', market_history_response['history'][-1])
print()

# and then also look at the most current view of the orderbook
market_history_params = {'ticker':ticker,
                            'depth': 30
                                }
orderbook_response = exchange_client.get_orderbook(**market_history_params)

print('keys:', orderbook_response.keys())
print()
print('orderbook object:', orderbook_response)
print()

HttpError: HttpError(404 Not Found)

In [ ]:
# Now let's suppose we wanted to place a trade on one of these markets... 
# to do so, we would first want to check out available balance...

current_balance = exchange_client.get_balance()
current_balance

HttpError: HttpError(401 Unauthorized)

In [ ]:
# Now that you have some balance, you might want to see how your current positions are doing...

positions_params = {'limit': None,
                        'cursor': None,
                        'settlement_status': None,
                        'ticker': None,
                        'event_ticker': None}

current_position = exchange_client.get_positions(**positions_params)
current_position

HttpError: HttpError(401 Unauthorized)

In [ ]:
# seems like some of your recent orders had been filled. To check on those we use the get_positions endpoint ...

fills_params = {'ticker':None,
                    'order_id':None,
                    'min_ts':None,
                    'max_ts':None,
                    'limit': None,
                    'cursor': None}

fills = exchange_client.get_fills(**fills_params)
fills

HttpError: HttpError(401 Unauthorized)

In [ ]:
# you may even want to check on some of your recent positions settled...

settlement_params = {'limit': None,
                        'cursor': None}

settlements = exchange_client.get_portfolio_settlements(**settlement_params)
settlements

HttpError: HttpError(401 Unauthorized)

In [ ]:
# Now onto placing an order...
# There are many different ways to think about placing orders at Kalshi. 
# The following param examples will walk through some of those

# Limit buy order for 10 units at 30c No on GDPW-22-A3

ticker = 'TESTING-5'

order_params = {'ticker':ticker,
                    'client_order_id':str(uuid.uuid4()),
                    'type':'limit',
                    'action':'buy',
                    'side':'no',
                    'count':10,
                    'yes_price':None, # yes_price = 100 - no_price
                    'no_price':30, # no_price = 100 - yes_price
                    'expiration_ts':None,
                    'sell_position_floor':None,
                    'buy_max_cost':None}

exchange_client.create_order(**order_params)

# EQUIVALENTLY, because buying No is equivalent to selling yes...

# order_params = {'ticker':ticker,
#                     'client_order_id':str(uuid.uuid4()),
#                     'type':'limit',
#                     'action':'sell',
#                     'side':'yes',
#                     'count':10,
#                     'yes_price':None, # yes_price = 100 - no_price
#                     'no_price':30, # no_price = 100 - yes_price
#                     'expiration_ts':None,
#                     'sell_position_floor':None,
#                     'buy_max_cost':None}

# exchange_client.create_order(**order_params)


# # Market sell order for 12 units Yes on GDPW-22-A3, without flipping position

# order_params = {'ticker':ticker,
#                     'client_order_id':str(uuid.uuid4()),
#                     'type':'market',
#                     'action':'sell',
#                     'side':'yes',
#                     'count':12,
#                     'yes_price':1,
#                     'no_price':None,
#                     'expiration_ts':None,
#                     'sell_position_floor':0,
#                     'buy_max_cost':None}

# exchange_client.create_order(**order_params)

# # EQUIVALENTLY, because buying No is equivalent to selling yes...

# order_params = {'ticker':ticker,
#                     'client_order_id':str(uuid.uuid4()),
#                     'type':'market',
#                     'action':'buy',
#                     'side':'no',
#                     'count':12,
#                     'yes_price':1,
#                     'no_price':None,
#                     'expiration_ts':None,
#                     'sell_position_floor':0,
#                     'buy_max_cost':None}

# exchange_client.create_order(**order_params)             


{'ticker': 'TESTING-5', 'client_order_id': 'c722356e-137d-431e-be87-086fcdefd458', 'side': 'no', 'action': 'buy', 'count': 10, 'type': 'limit', 'no_price': 30}


HttpError: HttpError(401 Unauthorized)